In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [2]:
hw121 = pd.read_csv('homework_12.1.csv', index_col=0)
hw121

,Y,Time,Group
0,-0.232900,-1.193204,0
1,2.848846,-1.607748,1
2,0.550209,-0.269793,0
3,2.198280,7.743730,0
4,4.111044,-4.244359,1
...,...,...,...
9995,5.188977,2.930515,0
9996,3.557366,0.275730,0
9997,2.208786,8.377715,0
9998,2.593101,-4.406393,0


In [5]:
treat = hw121[hw121['Group'] == 1]
control = hw121[hw121['Group'] == 0]

In [7]:
treat_1 = treat[treat['Time'] > 0]
treat_0 = treat[treat['Time'] <= 0]
control_1 = control[control['Time'] > 0]
control_0 = control[control['Time'] <= 0]

In [8]:
mean_treat_1 = np.mean(treat_1['Y'])
mean_treat_0 = np.mean(treat_0['Y'])
mean_control_1 = np.mean(control_1['Y'])
mean_control_0 = np.mean(control_0['Y'])

diff_in_diff = (mean_treat_1 - mean_treat_0) - (mean_control_1 - mean_control_0)
diff_in_diff

np.float64(0.9440103783528997)

-----------------------------------------------------------------------------

In [19]:
hw122 = pd.read_csv('homework_12.2.csv', index_col=0)
hw122

,Y,Time,Group
0,0.283614,-3.614416,0
1,3.181495,-3.064196,0
2,1.401619,-4.109261,0
3,2.019632,-0.706696,1
4,1.032628,-1.806227,0
...,...,...,...
19995,2.201824,-4.105641,1
19996,3.049469,-9.385249,0
19997,2.990919,9.686635,0
19998,1.824904,8.790771,0


In [24]:
hw122['Group x Time'] = hw122['Group'] * hw122['Time']
before_0 = hw122[hw122['Time'] < 0]
before_0

,Y,Time,Group,Group x Time
0,0.283614,-3.614416,0,-0.000000
1,3.181495,-3.064196,0,-0.000000
2,1.401619,-4.109261,0,-0.000000
3,2.019632,-0.706696,1,-0.706696
4,1.032628,-1.806227,0,-0.000000
...,...,...,...,...
19993,0.866278,-2.231896,0,-0.000000
19994,1.544645,-4.772292,0,-0.000000
19995,2.201824,-4.105641,1,-4.105641
19996,3.049469,-9.385249,0,-0.000000


In [25]:
sm.OLS(before_0['Y'], sm.add_constant(before_0[['Group', 'Time', 'Group x Time']])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.324
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     1591.
Date:                Fri, 08 Aug 2025   Prob (F-statistic):               0.00
Time:                        10:35:49   Log-Likelihood:                -13981.
No. Observations:                9944   AIC:                         2.797e+04
Df Residuals:                    9940   BIC:                         2.800e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.9996      0.022     45.154      0.000       0.956       1.043
Group            1.9510      0.050     38.685      0.000       1.852       2.050
Time             0.0016      0.004      0.427      0.669      -0.006       0.009
Group x Time     0.0464      0.009      5.329      0.000       0.029       0.063
==============================================================================
Omnibus:                        2.708   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.258   Jarque-Bera (JB):                2.713
Skew:                          -0.024   Prob(JB):                        0.257
Kurtosis:                       3.065   Cond. No.                         31.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

------------------------------------------------------------------------------

In [26]:
def generate_ar1(n, rho, std=1.0):
    sigma_eps = std * np.sqrt(1 - rho ** 2)
    x = np.zeros(n)
    eps = np.random.normal(0, sigma_eps, n)
    for t in range(1, n):
        x[t] = rho * x[t - 1] + eps[t]
    return x

n = 10000
rho = 0.8
n_trials = 1000
beta_estimates = []

for _ in range(n_trials):
    X = generate_ar1(n, rho)
    err = generate_ar1(n, rho)
    Y = 2 * X + err
    
    X_with_const = sm.add_constant(X)
    model = sm.OLS(Y, X_with_const).fit()
    beta_estimates.append(model.params[1]) # coefficient for X

beta_estimates = np.array(beta_estimates)
empirical_std = np.std(beta_estimates)
print(empirical_std)

0.022013436220025294


---------------------------------------------------------------------------

In [15]:
np.random.seed(0)
X1 = np.random.normal(0, 1, 1000)
X2 = np.random.normal(0, 1, 1000) + X1
X3 = np.random.normal(0, 1, 1000) + 2 * X2

In [16]:
data = {'X1': X1, 'X2': X2, 'X3': X3}
df = pd.DataFrame(data)

X = df[['X1', 'X2', 'X3']]
X = sm.add_constant(X)

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

  feature        VIF
0   const   1.005335
1      X1   1.978374
2      X2  10.108729
3      X3   8.989002
